In [5]:
# Instalación de librerías
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [6]:
#Adaptado por  Víctor Viera B.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#biblioteca de extracción de características del texto
from sklearn.feature_extraction.text import CountVectorizer
#para medir la precisión del modelo, matriz de confusión
from sklearn.metrics import confusion_matrix
#divide el conjunto de datos de entrenamiento
from sklearn.model_selection import train_test_split
#Modelo de Naive Bayes, El clasificador multinomial Naive Bayes es adecuado para la clasificación
#con características discretas (por ejemplo, recuento de palabras para la clasificación de texto)
from sklearn.naive_bayes import MultinomialNB

In [7]:
datos= pd.read_csv('./assets/Tweets.csv')

In [8]:
datos

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [9]:
X=datos['text']

In [10]:
datos['sentiment'] = np.where(datos['sentiment']=='neutral', 1, datos['sentiment'])
datos['sentiment'] = np.where(datos['sentiment']=='negative', 2, datos['sentiment'])
datos['sentiment'] = np.where(datos['sentiment']=='positive', 3, datos['sentiment'])


# Keep the original sentiment if it's not 'neutral'


#datos.rename(columns={0:'neutral',1:'negative',2:'positive'}, inplace=True)

In [11]:
datos

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",1
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,2
2,088c60f138,my boss is bullying me...,bullying me,2
3,9642c003ef,what interview! leave me alone,leave me alone,2
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",2
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,2
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",2
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,3
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,3


In [12]:
datos['text'] = datos['text'].fillna('')  # remplaza los  nullos con vacíos
datos['sentiment'] = datos['sentiment'].fillna('0')

/var/folders/kr/066v09l56c9_4l91brw4fwdc0000gn/T/ipykernel_3468/566179174.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos['sentiment'] = datos['sentiment'].fillna('0')


In [13]:
datos['sentiment'].value_counts()

sentiment
1    11118
3     8582
2     7781
Name: count, dtype: int64

In [14]:
#civde el conjunto de datos en entranemiento(X_train,X_test) y prueba(X_test,Y_test)
X_train, X_test, Y_train, Y_test = train_test_split(datos['text'],
                                                    datos['sentiment'],
                                                    random_state=42)

In [15]:
datos[["sentiment"]]


,sentiment
0,1
1,2
2,2
3,2
4,2
...,...
27476,2
27477,2
27478,3
27479,3


In [16]:
# extrear las características,aquí se convierten los textos a números con vector
#para aprender las dimensiones y características de los textos
vectorizer = CountVectorizer(ngram_range=(1, 2)).fit(X_train)
#se conviente a vectorización numérica, se trabaja con vectores numéricos, no con letras
X_train_vectorized = vectorizer.transform(X_train)


In [17]:
#se acomodan los datos para introducirlos
#el parámetro alfa es lo que se conoce como hiperparámetro; es decir,
#un parámetro que controla la forma del modelo en sí
#El suavizado de Laplace es una técnica utilizada para evitar problemas de probabilidad nula en el modelo de Naive Bayes.
#puede influir en el rendimiento y la generalización del clasificador.(laplace)
model = MultinomialNB(alpha=0.1)
#Ocurre la matemática "Bayes"
Modelo=model.fit(X_train_vectorized, Y_train)

In [18]:
#calcula la exactitud del modelo, realizo la predicción, y se convierte los textos a vector(n)
predictions = model.predict(vectorizer.transform(X_train))

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(predictions,Y_train)

0.986608442503639

In [20]:
from sklearn.metrics import classification_report
print(classification_report(Y_train,predictions))

              precision    recall  f1-score   support

           1       0.99      0.98      0.99      8331
           2       0.98      0.99      0.99      5824
           3       0.98      0.99      0.99      6455

    accuracy                           0.99     20610
   macro avg       0.99      0.99      0.99     20610
weighted avg       0.99      0.99      0.99     20610



In [21]:
#cómo se comporta de acuerdo a la matriz de confusión
#la diagonal principal indica los aciertos, los datos equivocados (3,11)
confusion_matrix(predictions, Y_train)

array([[8134,   24,   26],
       [  82, 5791,   20],
       [ 115,    9, 6409]])

In [22]:
# auí realizamos una prueba con texto nuevo
model.predict(vectorizer.transform(
    [
        "Thank you, ABC. Can you also share your LinkedIn profile? As you are a good at programming at pyhthon, would be willing to see your personal/college projects.",
        "Hi y’all, We have a Job Openings in the positions of software engineer, IT officer at ABC Company.Kindly, send us your resume and the cover letter as soon as possible if you think you are an eligible candidate and meet the criteria.",
        "Dear ABC, Congratulations! You have been selected as a SOftware Developer at XYZ Company. We were really happy to see your enthusiasm for this vision and mission. We are impressed with your background and we think you would make an excellent addition to the team.",
    ]))

array([3, 3, 3])

In [23]:
#texto de ejemplo
model.predict(vectorizer.transform(
    [
        "congratulations, you became today's lucky winner",
        "1-month unlimited calls offer Activate now",
        "Ram wants your phone number",

    ]))

array([3, 3, 2])

In [24]:
#calcula la exactitud del modelo, realizo la predicción, y se convierte los textos a vector(n)
predictionsPrueba = model.predict(vectorizer.transform(X_test))

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(predictionsPrueba,Y_test)

0.5983117450152816

In [26]:
#cómo se comporta de acuerdo a la matriz de confusión
#la diagonal principal indica los aciertos, los datos equivocados (3,11)
confusion_matrix(predictionsPrueba, Y_test)

array([[1533,  615,  582],
       [ 640, 1191,  158],
       [ 614,  151, 1387]])

In [27]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictionsPrueba))

              precision    recall  f1-score   support

           1       0.56      0.55      0.56      2787
           2       0.60      0.61      0.60      1957
           3       0.64      0.65      0.65      2127

    accuracy                           0.60      6871
   macro avg       0.60      0.60      0.60      6871
weighted avg       0.60      0.60      0.60      6871

